In [135]:
from collections import defaultdict
import sys
import os
import csv
import shutil
sys.path.append("/mnt/c/Users/gweld/sidewalk/sidewalk_ml/")
sys.path.append("/mnt/c/Users/gweld/sidewalk/sidewalk_ml/pytorch_pretrained")

from PIL import Image, ImageDraw, ImageFont, ImageColor
import numpy as np

import GSVutils.utils

from GSVutils.point import Point as Point
from GSVutils.pano_feats import Pano as Pano
from GSVutils.pano_feats import Feat as Feat
from GSVutils.clustering import non_max_sup
from GSVutils.precision_recall import precision_recall, partition_based_on_correctness

import torchvision
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

from TwoFileFolder import TwoFileFolder
from resnet_extended2 import extended_resnet18

GSV_IMAGE_HEIGHT = GSVutils.utils.GSV_IMAGE_HEIGHT
GSV_IMAGE_WIDTH  = GSVutils.utils.GSV_IMAGE_WIDTH

In [78]:
label_from_int = ('Curb Cut', 'Missing Cut', 'Obstruction', 'Sfc Problem')


In [79]:
sample_ground_truth = '/mnt/c/Users/gweld/sidewalk/sidewalk_ml/ground_truth/cv_output.csv'

path_to_gsv_scrapes  = "/mnt/f/scrapes_dump/"
#pano_db_export = '/mnt/c/Users/gweld/sidewalk/minus_onboard.csv'
pano_db_export = sample_ground_truth

In [80]:
# get panos
panos=set()

with open(sample_ground_truth) as csvfile:
    reader = csv.reader(csvfile)
    reader.next()
    for line in reader:
        pano_id=line[0][1:-1]
        panos.add(pano_id)

print panos

set(['QVBzWhGL0UYQAlOYe_EByA', 'tfbLvp9AXICPy1rVrevJIA', 'RFETpkMhl9_Ip91-iVFhLg', 's8GEb88vkcN3n7L9XSCOEw', 'UllByZTAKraVZUR5AsaTQw', 'xoJcAPI20GkVZhZEMXwe6A', 'cCSnGSMg3xL8X4hvjfie1w', 'Tlw9RExZSh3cezUjcMQDnw', '1R8sTrcpWMTmfOtH4Ima0A', 'Btx_jn4wzlB7lCJznodB8g', 'iTwDUyGnEjJleE6wMkbhXw', '273jMbv01qXmghYoMcrbnA', 'betttaqtbxoxAenR4qXzaQ', 'oelHXIPFCxM-GrqjXJ_Rjw', 'riixORAIsHjNKzXzO6ZiHQ', 'YRij4tm47MusTu-9BGYa1A', 'D1d8_1zf4MK2njkQRmTGZw', 'QDsRGDFJ8NDvjuMLLtQoJw'])


In [88]:
def convert_to_real_coords(sv_x, sv_y, pano_yaw_deg):
    x = ((float(pano_yaw_deg) / 360) * GSV_IMAGE_WIDTH + sv_x) % GSV_IMAGE_WIDTH
    y = GSV_IMAGE_HEIGHT / 2 - sv_y

    return int(x), int(y)


def convert_to_sv_coords(x, y, pano_yaw_deg):
    sv_x = x - (float(pano_yaw_deg)/360 * GSV_IMAGE_WIDTH)
    sv_y = GSV_IMAGE_HEIGHT / 2 - y

    if sv_x < 0: sv_x += GSV_IMAGE_WIDTH 

    return int(sv_x), int(sv_y)


In [89]:
def get_ground_truth(pano_id, true_pano_yaw_deg):
    ''' returns dict of str coords mapped to int label '''
    labels = {}
    with open(pano_db_export, 'r') as csvfile:
        reader = csv.reader(csvfile)

        for row in reader:
            if row[0][1:-1] != pano_id:
                continue
            x, y = int(row[1]), int(row[2])
            label = int(row[3])-1 # compensate for 1-indexing
            photog_heading = float(row[4])

            pano_yaw_deg = 180 - photog_heading

            x, y = convert_to_real_coords(x, y, pano_yaw_deg)
            x, y = convert_to_sv_coords(x, y, true_pano_yaw_deg)

            # ignore other labels 
            if label not in range(4):
                continue

            labels["{},{}".format(x,y)] = label
    return labels

In [122]:
# get points for each pano
ground_truths = {}
for pano_id in panos:
    print "Getting ground truth for pano {}".format(pano_id)
    pano_xml_path   = os.path.join(path_to_gsv_scrapes, pano_id[:2], pano_id + ".xml")
    try:
        true_pano_yaw_deg = GSVutils.utils.extract_panoyawdeg(pano_xml_path)
    except IOError as e:
        print "skipping bad XML for pano {}".format(pano_id)
        continue

    gt = get_ground_truth(pano_id, true_pano_yaw_deg)

    print "\tFound {} ground truth labels.".format(len(gt))
    ground_truths[pano_id] = gt
    


Getting ground truth for pano QVBzWhGL0UYQAlOYe_EByA
	Found 5 ground truth labels.
Getting ground truth for pano tfbLvp9AXICPy1rVrevJIA
	Found 47 ground truth labels.
Getting ground truth for pano RFETpkMhl9_Ip91-iVFhLg
	Found 3 ground truth labels.
Getting ground truth for pano s8GEb88vkcN3n7L9XSCOEw
	Found 9 ground truth labels.
Getting ground truth for pano UllByZTAKraVZUR5AsaTQw
	Found 24 ground truth labels.
Getting ground truth for pano xoJcAPI20GkVZhZEMXwe6A
skipping bad XML for pano xoJcAPI20GkVZhZEMXwe6A
Getting ground truth for pano cCSnGSMg3xL8X4hvjfie1w
	Found 12 ground truth labels.
Getting ground truth for pano Tlw9RExZSh3cezUjcMQDnw
	Found 7 ground truth labels.
Getting ground truth for pano 1R8sTrcpWMTmfOtH4Ima0A
	Found 9 ground truth labels.
Getting ground truth for pano Btx_jn4wzlB7lCJznodB8g
	Found 7 ground truth labels.
Getting ground truth for pano iTwDUyGnEjJleE6wMkbhXw
	Found 6 ground truth labels.
Getting ground truth for pano 273jMbv01qXmghYoMcrbnA
	Found 8 gro

In [124]:
def annotate(img, pano_yaw_deg, coords, label, color, show_coords=True, box=None):
    """ takes in an image object and labels it at specified coords
        translates streetview coords to pixel coords
        if given a box, marks that box around the label
    """
    sv_x, sv_y = coords
    x = ((float(pano_yaw_deg) / 360) * GSV_IMAGE_WIDTH + sv_x) % GSV_IMAGE_WIDTH
    y = GSV_IMAGE_HEIGHT / 2 - sv_y

    if show_coords: label = "{},{} {}".format(sv_x, sv_y, label)

    # radius for dot
    r = 20
    draw = ImageDraw.Draw(img)
    draw.ellipse((x - r, y - r, x + r, y + r), fill=color)
    if box is not None:
        half_box = box/2
        p1 = (x-half_box, y-half_box)
        p2 = (x+half_box, y+half_box)
        draw.rectangle([p1,p2], outline=color)

    font  = ImageFont.truetype("roboto.ttf", 60, encoding="unic")
    draw.text((x+r+10, y), label, fill=color, font=font)

In [128]:
def show_predictions_on_image(pano_root, predictions, out_img, show_coords=True, show_box=False):
    ''' annotates an image with a dict of string coordinates and labels
        if ground truth: also gets the ground truth and displays that as well '''
    pano_img_path   = pano_root + ".jpg"
    pano_xml_path   = pano_root + ".xml"
    pano_depth_path = pano_root + ".depth.txt"
    pano_yaw_deg    = GSVutils.utils.extract_panoyawdeg(pano_xml_path)

    img = Image.open(pano_img_path)

    def annotate_batch(predictions):
        count = 0
        for coords, prediction in predictions.iteritems():
            sv_x, sv_y = map(float, coords.split(','))

            if show_box:
                x = ((float(pano_yaw_deg) / 360) * GSV_IMAGE_WIDTH + sv_x) % GSV_IMAGE_WIDTH
                y = GSV_IMAGE_HEIGHT / 2 - sv_y
                try:
                    box = GSVutils.utils.predict_crop_size(x, y, GSV_IMAGE_WIDTH, GSV_IMAGE_HEIGHT, pano_depth_path)
                except:
                    print "Couldn't get crop size... skipping box"
                    box = None
            else: box = None

            label = str(label_from_int[prediction])
            if prediction == 0:
                color = ImageColor.getrgb('green')
            if prediction == 1:
                color = ImageColor.getrgb('red')
            if prediction == 2:
                color = ImageColor.getrgb('blue')
            if prediction == 3:
                color = ImageColor.getrgb('purple')
        
            #print "Found a {} at ({},{})".format(label, sv_x, sv_y)
            annotate(img, pano_yaw_deg, (sv_x, sv_y), label, color, show_coords, box)
            count += 1
        return count

    pred = annotate_batch(predictions)

    img.save(out_img)
    print "Marked {} ground truth labels on {}.".format(pred, out_img)

    return


In [131]:
def batch_visualize_preds(outdir):
    count = 0
    for pano_id in ground_truths:
        print "Annotating {}".format(pano_id)
        predictions = ground_truths[pano_id]

        outfile = os.path.join(outdir, pano_id+'.jpg')

        pano_root = os.path.join(path_to_gsv_scrapes, pano_id[:2], pano_id)
        show_predictions_on_image(pano_root, predictions, outfile, show_coords=False, show_box=True)
        count += 1

        #if count > 1: break
    print "Wrote predictions for {} panos to {}".format(count, outdir)
    return

In [132]:
batch_visualize_preds('output/')

Annotating QVBzWhGL0UYQAlOYe_EByA
Marked 5 ground truth labels on output/QVBzWhGL0UYQAlOYe_EByA.jpg.
Annotating tfbLvp9AXICPy1rVrevJIA
Marked 47 ground truth labels on output/tfbLvp9AXICPy1rVrevJIA.jpg.
Annotating RFETpkMhl9_Ip91-iVFhLg
Marked 3 ground truth labels on output/RFETpkMhl9_Ip91-iVFhLg.jpg.
Annotating UllByZTAKraVZUR5AsaTQw
Marked 24 ground truth labels on output/UllByZTAKraVZUR5AsaTQw.jpg.
Annotating s8GEb88vkcN3n7L9XSCOEw
Marked 9 ground truth labels on output/s8GEb88vkcN3n7L9XSCOEw.jpg.
Annotating cCSnGSMg3xL8X4hvjfie1w
Marked 12 ground truth labels on output/cCSnGSMg3xL8X4hvjfie1w.jpg.
Annotating Tlw9RExZSh3cezUjcMQDnw
Marked 7 ground truth labels on output/Tlw9RExZSh3cezUjcMQDnw.jpg.
Annotating YRij4tm47MusTu-9BGYa1A
Marked 7 ground truth labels on output/YRij4tm47MusTu-9BGYa1A.jpg.
Annotating Btx_jn4wzlB7lCJznodB8g
Marked 7 ground truth labels on output/Btx_jn4wzlB7lCJznodB8g.jpg.
Annotating oelHXIPFCxM-GrqjXJ_Rjw
Marked 11 ground truth labels on output/oelHXIPFCxM-Gr

In [123]:
ground_truths['QVBzWhGL0UYQAlOYe_EByA']

{'1,-345': 1, '2006,-81': 0, '3390,-101': 3, '3901,-318': 3, '4664,-655': 1}

In [139]:
# get unmarked_panos_for_comparison
for pano_id in ground_truths:
    pano_path = os.path.join(path_to_gsv_scrapes, pano_id[:2], pano_id + ".jpg")
    dest_path = os.path.join('output', pano_id +'_unmarked.jpg')
    shutil.copyfile(pano_path, dest_path)